In [2]:
import lasio
import pandas as pd
import os
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from sklearn.neighbors import RadiusNeighborsClassifier
from scipy.spatial import distance

        

In [10]:
# Создание списка с названиями скважин

targ_form = pd.read_csv('FieldWTPS1.csv',sep=';')
targ_form = targ_form.groupby(['Well','Surface']).aggregate({'MD':'mean'})
files = []
length = targ_form['MD'].size
for i in range(length):
    files.append( targ_form['MD'].index[i][0])
files = set(files)

In [50]:
# Подготовка данных каротажа

for i_file in files:
    try:
        cor = lasio.read('path/' + i_file +'.las')
        SP = pd.DataFrame({'DEPT':cor['DEPT'], 'SP':cor['SP']})
        SP = SP.fillna(0)  
        SP = SP.query("SP != 0 & SP != 9999 & SP != (-9999)")
        if targ_form.loc[i_file.split('.')[0]].nunique()[0] ==2:
            top=targ_form.loc[(i_file.split('.')[0],'Top_Georg')][0]
            bottom = targ_form.loc[(i_file.split('.')[0],'U1-3_bot')][0]
        else:
            top=targ_form.loc[(i_file.split('.')[0],'Top_Georg')][0]
            bottom = top+100
        SP = SP.query("DEPT >= @top  & DEPT <= @bottom ")
        if SP.size>0:
            typic = SP.rename(columns={'DEPT':'Y','SP':'X'})
            typic.Y = (typic.Y-typic.Y.min())*55/(typic.Y.max()-typic.Y.min())
            typic.X = (typic.X-typic.X.min())*1/(typic.X.max()-typic.X.min())
            f = interpolate.interp1d(typic.Y.values, typic.X.values)
            ynew = np.arange(0,55,1)
            xnew = f(ynew)
            corotazh_total = pd.DataFrame({'Y':ynew,'X':xnew[::-1]})
            corotazh_total.to_csv('OutData2/'+i_file.split('.')[0]+'.csv')
    except Exception:  
        pass
    

Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.


In [ ]:
# Подготовка данных типовых кривых

files = os.listdir(path ="typical/CSV")
for i in files:
    typic = pd.read_csv('typical/CSV/' + i, sep=';')
    typic.Y = (typic.Y-typic.Y.min())*55/(typic.Y.max()-typic.Y.min())
    typic.X = (typic.X-typic.X.min())*1/(typic.X.max()-typic.X.min())
    f = interpolate.interp1d(typic.Y.values, typic.X.values)
    xnew = f(np.arange(0,55,1))
    corotazh_total = pd.DataFrame({'Y':np.arange(0,55,1),'X':xnew})
    corotazh_total.to_csv('typical/reb/' + i)



In [5]:
# Подготовка данных типовых кривых для обучения модели  

typ_cur = pd.DataFrame()
files = os.listdir(path ="typical/reb")
for i_file in files :
    X = i_file.split('.')[0]
    curve = pd.read_csv('reb/'+i_file,index_col=0)
    curve = curve.rename(columns={'X': X})
    typ_cur = pd.concat([typ_cur,curve[[X]]],axis=1)
typ_cur = typ_cur.T
typ_cur = typ_cur.reset_index()
typ_cur = typ_cur.rename(columns={'index': 'Y'})
typ_cur = typ_cur.sort_values(by='Y')

,Y,0,1,2,3,4,5,6,7,8,...,45,46,47,48,49,50,51,52,53,54
0,1,0.941176,0.933779,0.926381,0.904337,0.852050,0.786988,0.741533,0.716578,0.696322,...,0.481283,0.608734,0.690434,0.750446,0.825312,0.885918,0.909091,0.929717,0.943808,0.957898
1,10,1.000000,0.986591,0.973181,0.949053,0.896752,0.828623,0.343992,0.193551,0.067680,...,0.700465,0.713627,0.738199,0.757493,0.770185,0.782877,0.802715,0.823768,0.835230,0.846693
2,2,1.000000,0.983829,0.974505,0.968469,0.962432,0.956396,0.944745,0.931987,0.918407,...,0.648074,0.754534,0.824871,0.865617,0.884543,0.904411,0.919420,0.931822,0.944223,0.959161
3,3,1.000000,0.988627,0.984806,0.980984,0.973403,0.964620,0.958831,0.954659,0.950488,...,0.124730,0.365750,0.605980,0.704853,0.819626,0.866852,0.883875,0.919682,0.939372,0.957330
4,4,1.000000,0.981869,0.949081,0.937895,0.926949,0.895561,0.858628,0.797989,0.726010,...,0.634088,0.703019,0.762843,0.815214,0.858639,0.894795,0.926071,0.937600,0.950035,0.981285
5,5,1.000000,0.989366,0.978732,0.958952,0.930129,0.895149,0.860953,0.817305,0.770703,...,0.718077,0.761316,0.803265,0.842663,0.886967,0.922969,0.936642,0.948475,0.956317,0.964159
6,6,0.917694,0.903668,0.889643,0.863485,0.847392,0.830410,0.802041,0.782300,0.748356,...,0.805010,0.840490,0.861352,0.891731,0.907941,0.926699,0.946738,0.957400,0.968061,0.982324
7,7,0.784512,0.730112,0.396289,0.236306,0.181170,0.138761,0.095124,0.079482,0.063837,...,0.156136,0.222066,0.331914,0.473891,0.607013,0.690655,0.849231,0.897076,0.962082,0.992764
8,8,0.947464,0.920373,0.903961,0.868142,0.805510,0.420559,0.262986,0.181635,0.136478,...,0.684683,0.731991,0.779298,0.823719,0.860862,0.891594,0.926636,0.946736,0.974976,0.987488
9,9,0.910676,0.892482,0.864424,0.347831,0.175215,0.117641,0.082172,0.047697,0.029123,...,0.851329,0.866256,0.880904,0.907570,0.922216,0.935196,0.952190,0.970975,0.980650,0.990325


In [6]:


X_train = typ_cur.drop(columns=['Y'])
Y_train = typ_cur[['Y']]




In [7]:
# Задаём собственную метрику

def DTW(a, b):   
    an = a.size
    bn = b.size
    pointwise_distance = distance.cdist(a.reshape(-1,1),b.reshape(-1,1))
    cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
    cumdist[0,0] = 0

    for ai in range(an):
        for bi in range(bn):
            minimum_cost = np.min([cumdist[ai, bi+1],
                                   cumdist[ai+1, bi],
                                   cumdist[ai, bi]])
            cumdist[ai+1, bi+1] = pointwise_distance[ai,bi] + minimum_cost

    return cumdist[an, bn]





In [8]:
# Обучение модели ближайших соседей. Вбран вариант с заданым радиусом, 
# для отсеивания каротажей не принадлежащих ни к одному классу.
# Радиус подобран эксперементальным путем на основе данных ПС каротажа целевого пласта.

rnc = RadiusNeighborsClassifier(n_neighbors=10, metric=DTW, weights='distance', radius=6.3)
rnc.fit(X_train,Y_train)

C:\ProgramData\Anaconda3\envs\untitled6\lib\site-packages\sklearn\neighbors\_classification.py:409: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  SupervisedIntegerMixin.fit(self, X, y)


RadiusNeighborsClassifier(metric=<function DTW at 0x00000205DF23EAF8>,
                          radius=5.5, weights='distance')

In [ ]:
# Проверка работоспособности модели для подбора радиуса (К данным ПС каротажа целевого пласта добавлены 
# кривые не относящиеся ни к одному из 10 классов. Предполагается, что радиус подобран верно, если все эти кривые, 
# и только они, окажутся в списке error_files)

files=os.listdir(path='HW/OutData2')
error_files=[]
for i in files:
    try:
        cor = pd.read_csv('OutData2/'+ i)
        test = cor[['X']].head(55)
        test = test.T
        print(rnc.predict_proba(test))
        print(rnc.predict(test))
    except Exception:
        error_files.append(i)
        
error_files


In [12]:
# Визуальная проверка работы модели

i_file = '360'
cor = lasio.read('path/' + i_file +'.las')
SP = pd.DataFrame({'DEPT':cor['DEPT'], 'SP':cor['SP']})
SP = SP.fillna(0)  
SP = SP.query("SP != 0 & SP != 9999 & SP != (-9999)")
if targ_form.loc[i_file.split('.')[0]].nunique()[0] ==2:
    top=targ_form.loc[(i_file.split('.')[0],'Top_Georg')][0]
    bottom = targ_form.loc[(i_file.split('.')[0],'U1-3_bot')][0]
else:
    top=targ_form.loc[(i_file.split('.')[0],'Top_Georg')][0]
    bottom = top+100
SP = SP.query("DEPT >= @top  & DEPT <= @bottom ")
if SP.size>0:
    typic = SP.rename(columns={'DEPT':'Y','SP':'X'})
    typic.Y = (typic.Y-typic.Y.min())*55/(typic.Y.max()-typic.Y.min())
    typic.X = (typic.X-typic.X.min())*1/(typic.X.max()-typic.X.min())
    f = interpolate.interp1d(typic.Y.values, typic.X.values)
    xnew = f(np.arange(0,55,1))
    test = pd.DataFrame({'0':xnew[::-1]})

test = test.T

SP = pd.read_csv('OutData2/'+i_file+'.csv',index_col=0 )
SP1 = pd.read_csv('typical/reb/'+rnc.predict(test)[0]+'.csv' )
x = SP.X
y = SP.Y
X = SP1.X
Y = SP1.Y
plt.plot(x, y, '-', X, Y, '-')
plt.show()

print(rnc.predict_proba(test))
print(rnc.predict(test))


Header section Parameter regexp=~P was not found.


ValueError: No neighbors found for test samples array([0], dtype=int64), you can try using larger radius, giving a label for outliers, or considering removing them from your dataset.

In [ ]:
# Алгоритм находящий на кривой каротажа участки соответствующие типовым кривым.
# Задано ограничение глубины от 1500м., так как нефтегазонасыщенные породы исследуемого региона
# залегают глубже данной отметки

i_file = '200R'
cor = lasio.read('path/' + i_file +'.las')
SP = pd.DataFrame({'Y':cor['DEPT'], 'X':cor['SP']})
SP = SP.fillna(0)
SP = SP.query("X != 0 & X != 9999 & X != (-9999) & Y > 1500 ")
top  = SP.Y.values[0]-10
max_dept  = SP.Y.values[-1]
bottom = top+100
clf_data = pd.DataFrame({'top':[], 'bottom':[], 'class':[]})
while bottom <= max_dept:
    try:
        top += 10
        bottom = top + 100
        typic = SP.query("Y >= @top & Y <= @bottom ")
        typic.Y = (typic.Y-typic.Y.min())*55/(typic.Y.max()-typic.Y.min())
        typic.X = (typic.X-typic.X.min())*1/(typic.X.max()-typic.X.min())
        f = interpolate.interp1d(typic.Y.values, typic.X.values)
        xnew = f(np.arange(0,55,1))
        test = pd.DataFrame({'0':xnew[::-1]})
        test = test.T
        try:
            prediction = rnc.predict(test)
            clf_data = pd.concat([clf_data,pd.DataFrame({'top':top, 'bottom':bottom, 'class':prediction})])
        except Exception:  
            pass
        
    except Exception:
        pass
        

top = max_dept - 100
bottom = max_dept
typic = SP.query("Y >= @top & Y <= @bottom ")
typic.Y = (typic.Y-typic.Y.min())*55/(typic.Y.max()-typic.Y.min())
typic.X = (typic.X-typic.X.min())*1/(typic.X.max()-typic.X.min())
f = interpolate.interp1d(typic.Y.values, typic.X.values)
xnew = f(np.arange(0,55,1))
test = pd.DataFrame({'0':xnew[::-1]})
test = test.T
try:
    prediction = rnc.predict(test)
    clf_data = pd.concat([clf_data,pd.DataFrame({'top':top, 'bottom':bottom, 'class':prediction})])
except Exception:  
    pass



In [ ]:
# Сохраняем полученный результат

clf_data = clf_data.reset_index()
clf_data = clf_data.drop(columns=['index'])
clf_data.to_excel('clf_data.xls')
